In [ ]:
import numpy as np
import os
import tensorflow as tf
import keras 
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Conv2D, BatchNormalization, Dropout, Flatten
from keras.layers import Activation, Reshape, Conv2DTranspose, UpSampling2D
from keras.optimizers import RMSprop

import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
def transpose_conv2d(x, output_space):
    return Conv2DTranspose(output_space, kernel_size=5,strides=2, padding='same',
                           kernel_initializer=tf.random_normal_initializer(mean=0.0,stddev=0.02))(x)

In [ ]:
input_images = "./apple.npy"

In [ ]:
data = np.load(input_images)

In [ ]:
data.shape

In [ ]:
data[4242]

In [ ]:
data = data/255
data = np.reshape(data,(data.shape[0], 28,28,1))
img_w, img_h = data.shape[1:3]
data.shape

In [ ]:
plt.imshow(data[4242,:,:,0], cmap='Greys')

In [ ]:
def discriminator_builder(depth=64, p=0.4):
    # Define input
    inputs = Input((img_w, img_h,1))
    
    # Convolutional layers
    conv1 = Conv2D(depth*1, 5, strides=2, padding='same', activation='relu')(inputs)
    conv1 = Dropout(p)(conv1)
    
    conv2 = Conv2D(depth*2, 5, strides=2, padding='same', activation='relu')(conv1)
    conv2 = Dropout(p)(conv2)
    
    conv3 = Conv2D(depth*4, 5, strides=2, padding='same', activation='relu')(conv2)
    conv3 = Dropout(p)(conv3)
    
    conv4 = Conv2D(depth*8, 5, strides=1, padding='same', activation='relu')(conv3)
    conv4 = Flatten()(Dropout(p)(conv4))
    
    output = Dense(1, activation='sigmoid')(conv4)
    
    model = Model(inputs=inputs, outputs=output)
    model.summary()
    
    return model
    

In [ ]:
discriminator = discriminator_builder()

In [ ]:
discriminator.compile(loss='binary_crossentropy', 
                      optimizer=RMSprop(lr=0.0008, decay=6e-8, clipvalue=1.0),
                      metrics=['accuracy'])

In [ ]:
def generator_builder(z_dim=100, depth=64, p=0.4):
    inputs = Input((z_dim,))
    
    # First dense layer
    dens1 = Dense(7*7*64)(inputs)
    dens1 = BatchNormalization(momentum=0.9)(dens1)
    dens1 = Activation(activation='relu')(dens1)
    dens1 = Reshape((7,7,64))(dens1)
    
    # De-convolutions layers
    conv1 = UpSampling2D()(dens1)
    conv1 = Conv2DTranspose(int(depth/2), kernel_size=5,padding='same',activation=None,)(conv1)
    conv1 = BatchNormalization(momentum=0.9)(conv1)
    conv1 = Activation(activation='relu')(conv1)
    
    conv2 = UpSampling2D()(conv1)
    conv2 = Conv2DTranspose(int(depth/4), kernel_size=5,padding='same',activation=None,)(conv2)
    conv2 = BatchNormalization(momentum=0.9)(conv2)
    conv2 = Activation(activation='relu')(conv2)
    
    conv3 = Conv2DTranspose(int(depth/8), kernel_size=5,padding='same',activation=None,)(conv2)
    conv3 = BatchNormalization(momentum=0.9)(conv3)
    conv3 = Activation(activation='relu')(conv3)
    
    output = Conv2D(1, kernel_size=5, padding='same', activation='sigmoid')(conv3)
    
    model = Model(inputs=inputs, outputs=output)
    model.summary()
    
    return model

In [ ]:
generator = generator_builder()

In [ ]:
def adversarial_builder(z_dim=100):
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    model.compile(loss='binary_crossentropy',
                 optimizer = RMSprop(lr=0.0004,decay=3e-8,clipvalue=1.0),
                 metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
adversarial_model = adversarial_builder()

In [ ]:
def make_trainable(net, val):
    net.trainable = val
    for l in net.layers:
        l.trainable = val

In [ ]:
data.shape

In [ ]:
def train(epochs=1000, batch=128):
    d_metrics = []
    a_metrics = []
    
    running_d_loss = 0
    running_d_acc = 0
    running_a_loss = 0
    running_a_acc = 0
    
    for i in range(epochs):
        
        if i%100 == 0:
            print(i)
        
        real_imgs = np.reshape(data[np.random.choice(data.shape[0],batch,replace=False)],(batch,28,28,1))
        fake_imgs = generator.predict(np.random.uniform(-1.0, 1.0, size=[batch, 100]))
        
        if i%10 == 0:
            real_imgs.shape
            
        x = np.concatenate((real_imgs,fake_imgs))
        y = np.ones([2*batch,1])
        y[batch:,:] = 0
        
        make_trainable(discriminator, True)
        
        d_metrics.append(discriminator.train_on_batch(x,y))
        running_d_loss += d_metrics[-1][0]
        running_d_acc += d_metrics[-1][1]
        
        make_trainable(discriminator, False)
        
        noise = np.random.normal(loc=0,scale=1,size=[batch, 100])
        y = np.ones([batch,1])

        a_metrics.append(adversarial_model.train_on_batch(noise,y)) 
        running_a_loss += a_metrics[-1][0]
        running_a_acc += a_metrics[-1][1]
        
        if (i+1)%500 == 0:

            print('Epoch #{}'.format(i+1))
            log_mesg = "%d: [D loss: %f, acc: %f]" % (i, running_d_loss/i, running_d_acc/i)
            log_mesg = "%s  [A loss: %f, acc: %f]" % (log_mesg, running_a_loss/i, running_a_acc/i)
            print(log_mesg)

            noise = np.random.uniform(-1.0, 1.0, size=[16, 100])
            gen_imgs = generator.predict(noise)

            plt.figure(figsize=(5,5))

            for k in range(gen_imgs.shape[0]):
                plt.subplot(4, 4, k+1)
                plt.imshow(gen_imgs[k, :, :, 0], cmap='gray')
                plt.axis('off')
                
            plt.tight_layout()
            plt.show()
    
    return a_metrics, d_metrics

In [ ]:
a_metrics_complete, d_metrics_complete = train(epochs=50)